In [ ]:
pip install datasets

In [ ]:
pip install transformers

In [ ]:
pip install -q google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = False)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from transformers import BertTokenizer, BertModel
import re
import google.generativeai as palm
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
from matplotlib.ticker import MaxNLocator
import sklearn.metrics as skmetrics
import seaborn as sns
import tqdm
import torch
from transformers import GPT2Model, GPT2Tokenizer

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Datasets/train_palm.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Datasets/test_palm.csv')

In [ ]:
train_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2,wall st. bear claw back black ( reuter ) reute...,"[-0.0052881585, 0.013725331, -0.010678303, 0.0..."
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,carlyl look toward commerci aerospac ( reuter ...,"[0.024525622, 0.008544347, 0.009066997, 0.0182..."
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2,oil economi cloud stock ' outlook ( reuter ) r...,"[-0.004074435, 0.01028465, -0.025315488, -0.02..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,2,iraq halt oil export main southern pipelin ( r...,"[-0.032672904, 0.0065261093, -0.031091025, -0...."
4,4,"Oil prices soar to all-time record, posing new...",2,"oil price soar all-tim record , pose new menac...","[-0.03617165, 0.011841308, -0.0106031075, -0.0..."


In [ ]:
test_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings
0,0,Fears for T N pension after talks Unions repre...,2,fear n pension talk union repres worker turner...,"[0.046950016, 0.004197339, -0.020274937, 0.020..."
1,1,The Race is On: Second Private Team Sets Launc...,3,race : second privat team set launch date huma...,"[-0.011750515, 0.023927508, -0.010048168, 0.01..."
2,2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,ky. compani win grant studi peptid ( ap ) ap -...,"[0.0010361855, -0.03565198, -0.0056499154, 0.0..."
3,3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,predict unit help forecast wildfir ( ap ) ap -...,"[0.034863107, 0.020880563, -0.04296487, 0.0427..."
4,4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,calif. aim limit farm-rel smog ( ap ) ap - sou...,"[0.0207801, -0.010970387, -0.0130679505, 0.033..."


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2Model.from_pretrained('gpt2')
i = 0
def get_gpt_embeddings(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    global i
    print(i)
    i+=1
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1).tolist()
    return embeddings[0]

train_df['Embeddings_GPT'] = train_df['processed_text'].apply(get_gpt_embeddings)
test_df['Embeddings_GPT'] = test_df['processed_text'].apply(get_gpt_embeddings)

Streaming output truncated to the last 5000 lines.
122600
122601
122602
122603
122604
122605
122606
122607
122608
122609
122610
122611
122612
122613
122614
122615
122616
122617
122618
122619
122620
122621
122622
122623
122624
122625
122626
122627
122628
122629
122630
122631
122632
122633
122634
122635
122636
122637
122638
122639
122640
122641
122642
122643
122644
122645
122646
122647
122648
122649
122650
122651
122652
122653
122654
122655
122656
122657
122658
122659
122660
122661
122662
122663
122664
122665
122666
122667
122668
122669
122670
122671
122672
122673
122674
122675
122676
122677
122678
122679
122680
122681
122682
122683
122684
122685
122686
122687
122688
122689
122690
122691
122692
122693
122694
122695
122696
122697
122698
122699
122700
122701
122702
122703
122704
122705
122706
122707
122708
122709
122710
122711
122712
122713
122714
122715
122716
122717
122718
122719
122720
122721
122722
122723
122724
122725
122726
122727
122728
122729
122730
122731
122732
122733
122734
1227

In [ ]:
train_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings,Embeddings_GPT
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2,wall st. bear claw back black ( reuter ) reute...,"[-0.0052881585, 0.013725331, -0.010678303, 0.0...","[-0.02057463862001896, -0.09361176192760468, -..."
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,carlyl look toward commerci aerospac ( reuter ...,"[0.024525622, 0.008544347, 0.009066997, 0.0182...","[0.25809192657470703, -0.39266201853752136, -0..."
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2,oil economi cloud stock ' outlook ( reuter ) r...,"[-0.004074435, 0.01028465, -0.025315488, -0.02...","[0.2334199994802475, -0.1681075245141983, -0.4..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,2,iraq halt oil export main southern pipelin ( r...,"[-0.032672904, 0.0065261093, -0.031091025, -0....","[0.010993259027600288, 0.06956441700458527, -0..."
4,4,"Oil prices soar to all-time record, posing new...",2,"oil price soar all-tim record , pose new menac...","[-0.03617165, 0.011841308, -0.0106031075, -0.0...","[4.616823571268469e-05, -0.0872439444065094, -..."


In [ ]:
test_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings,Embeddings_GPT
0,0,Fears for T N pension after talks Unions repre...,2,fear n pension talk union repres worker turner...,"[0.046950016, 0.004197339, -0.020274937, 0.020...","[0.00060757074970752, -0.21469300985336304, -0..."
1,1,The Race is On: Second Private Team Sets Launc...,3,race : second privat team set launch date huma...,"[-0.011750515, 0.023927508, -0.010048168, 0.01...","[0.4147956669330597, -0.13466626405715942, 0.1..."
2,2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,ky. compani win grant studi peptid ( ap ) ap -...,"[0.0010361855, -0.03565198, -0.0056499154, 0.0...","[-0.19595442712306976, -0.3179986774921417, -0..."
3,3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,predict unit help forecast wildfir ( ap ) ap -...,"[0.034863107, 0.020880563, -0.04296487, 0.0427...","[-0.11235450953245163, -0.02521243318915367, -..."
4,4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,calif. aim limit farm-rel smog ( ap ) ap - sou...,"[0.0207801, -0.010970387, -0.0130679505, 0.033...","[0.09054004400968552, -0.12668471038341522, -0..."


In [ ]:
# Step 1: Preprocess the text
def preprocess_text(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    return tokens

train_df['tokens'] = train_df['processed_text'].apply(preprocess_text)



# Step 2: Load the BERT model
model = BertModel.from_pretrained('bert-base-uncased')
i = 0
# Step 3: Input formatting and obtaining embeddings
def get_bert_embeddings(tokens):
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    token_type_ids = tokens['token_type_ids']
    global i
    print(i)
    i+=1

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # The output is a tuple containing the last-layer hidden states and the pooled output.
    # We'll use the pooled output, which is a fixed-size representation of the entire paragraph.
    embeddings = outputs[1][0].tolist()
    return embeddings

train_df['Embeddings_Bert'] = train_df['tokens'].apply(get_bert_embeddings)
test_df['tokens'] = test_df['processed_text'].apply(preprocess_text)
test_df['Embeddings_Bert'] = test_df['tokens'].apply(get_bert_embeddings)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8c9be011-4688-4989-8b2c-229c07e1ad4d)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 18699c94-54e2-41d1-ade5-e8647a75e46f)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b4fab002-666d-419e-bb74-c1fa4f3d3cac)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7672921c-0889-4076-85b1-5348420f0b6d)')' thrown while requesting HEAD https://huggingface.co/bert-

Streaming output truncated to the last 5000 lines.
122600
122601
122602
122603
122604
122605
122606
122607
122608
122609
122610
122611
122612
122613
122614
122615
122616
122617
122618
122619
122620
122621
122622
122623
122624
122625
122626
122627
122628
122629
122630
122631
122632
122633
122634
122635
122636
122637
122638
122639
122640
122641
122642
122643
122644
122645
122646
122647
122648
122649
122650
122651
122652
122653
122654
122655
122656
122657
122658
122659
122660
122661
122662
122663
122664
122665
122666
122667
122668
122669
122670
122671
122672
122673
122674
122675
122676
122677
122678
122679
122680
122681
122682
122683
122684
122685
122686
122687
122688
122689
122690
122691
122692
122693
122694
122695
122696
122697
122698
122699
122700
122701
122702
122703
122704
122705
122706
122707
122708
122709
122710
122711
122712
122713
122714
122715
122716
122717
122718
122719
122720
122721
122722
122723
122724
122725
122726
122727
122728
122729
122730
122731
122732
122733
122734
1227

In [ ]:
train_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings,Embeddings_GPT,tokens,Embeddings_Bert
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2,wall st. bear claw back black ( reuter ) reute...,"[-0.0052881585, 0.013725331, -0.010678303, 0.0...","[-0.02057463862001896, -0.09361176192760468, -...","[input_ids, token_type_ids, attention_mask]","[-0.6562738418579102, -0.27509886026382446, -0..."
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,carlyl look toward commerci aerospac ( reuter ...,"[0.024525622, 0.008544347, 0.009066997, 0.0182...","[0.25809192657470703, -0.39266201853752136, -0...","[input_ids, token_type_ids, attention_mask]","[-0.39169448614120483, -0.3573859930038452, -0..."
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2,oil economi cloud stock ' outlook ( reuter ) r...,"[-0.004074435, 0.01028465, -0.025315488, -0.02...","[0.2334199994802475, -0.1681075245141983, -0.4...","[input_ids, token_type_ids, attention_mask]","[-0.743872880935669, -0.5422914624214172, -0.9..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,2,iraq halt oil export main southern pipelin ( r...,"[-0.032672904, 0.0065261093, -0.031091025, -0....","[0.010993259027600288, 0.06956441700458527, -0...","[input_ids, token_type_ids, attention_mask]","[-0.8004133701324463, -0.5173243880271912, -0...."
4,4,"Oil prices soar to all-time record, posing new...",2,"oil price soar all-tim record , pose new menac...","[-0.03617165, 0.011841308, -0.0106031075, -0.0...","[4.616823571268469e-05, -0.0872439444065094, -...","[input_ids, token_type_ids, attention_mask]","[-0.7293984293937683, -0.44600093364715576, -0..."


In [ ]:
test_df.head()

,Unnamed: 0,text,label,processed_text,Embeddings,Embeddings_GPT,tokens,Embeddings_Bert
0,0,Fears for T N pension after talks Unions repre...,2,fear n pension talk union repres worker turner...,"[0.046950016, 0.004197339, -0.020274937, 0.020...","[0.00060757074970752, -0.21469300985336304, -0...","[input_ids, token_type_ids, attention_mask]","[-0.7051424980163574, -0.5113633275032043, -0...."
1,1,The Race is On: Second Private Team Sets Launc...,3,race : second privat team set launch date huma...,"[-0.011750515, 0.023927508, -0.010048168, 0.01...","[0.4147956669330597, -0.13466626405715942, 0.1...","[input_ids, token_type_ids, attention_mask]","[-0.556801438331604, -0.38825464248657227, -0...."
2,2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,ky. compani win grant studi peptid ( ap ) ap -...,"[0.0010361855, -0.03565198, -0.0056499154, 0.0...","[-0.19595442712306976, -0.3179986774921417, -0...","[input_ids, token_type_ids, attention_mask]","[-0.7478275299072266, -0.1924387365579605, -0...."
3,3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,predict unit help forecast wildfir ( ap ) ap -...,"[0.034863107, 0.020880563, -0.04296487, 0.0427...","[-0.11235450953245163, -0.02521243318915367, -...","[input_ids, token_type_ids, attention_mask]","[-0.5818230509757996, -0.15323442220687866, -0..."
4,4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,calif. aim limit farm-rel smog ( ap ) ap - sou...,"[0.0207801, -0.010970387, -0.0130679505, 0.033...","[0.09054004400968552, -0.12668471038341522, -0...","[input_ids, token_type_ids, attention_mask]","[-0.7275802493095398, -0.2375248819589615, -0...."


In [ ]:
train_df.to_csv('Train_Dataset_With_Embeddings.csv')

In [ ]:
test_df.to_csv('Test_Dataset_With_Embeddings.csv')

In [ ]:
cp /content/Train_Dataset_With_Embeddings.csv /content/drive/MyDrive/Datasets/

In [ ]:
cp /content/Test_Dataset_With_Embeddings.csv /content/drive/MyDrive/Datasets/